In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import torchvision
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

import torchvision.transforms as transforms
import torch
from torch.utils.data.sampler import Sampler
import numpy as np
import itertools
import random

from torchvision.datasets import SVHN, MNIST, FashionMNIST, CIFAR10, CelebA, Omniglot

In [3]:
root = "/scratch/gg2501/ML_Project/"

In [4]:
sys.path.append(os.path.join(root, 'flowgmm-public'))

In [5]:
import flow_ssl

In [6]:
data_dir = os.path.join(root, 'data')

In [7]:
svhn_dataset = SVHN(root=data_dir, split='train', download=True)
mnist_dataset = MNIST(root=data_dir, download=True)
fashionmnist_dataset = FashionMNIST(root=data_dir, download=True)
cifar_dataset = CIFAR10(root=data_dir, download=True)

Using downloaded and verified file: /scratch/gg2501/ML_Project/data/train_32x32.mat
Files already downloaded and verified


In [8]:
from torchvision.datasets import STL10, Food101, Caltech101, GTSRB, Flowers102
stl_dataset = STL10(root=data_dir, split='train', download=True)
food_dataset = Food101(root=data_dir, download = True)
# celeb_dataset = CelebA(root=data_dir, download = True)
flowers_dataset = Flowers102(root=data_dir, download = True)
caltech_dataset = Caltech101(root=data_dir, download = True)
german_sign_dataset = GTSRB(root=data_dir, download = True)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
class LabeledUnlabeledBatchSampler(Sampler):
    """Minibatch index sampler for labeled and unlabeled indices. 

    An epoch is one pass through the labeled indices.
    """
    def __init__(
            self, 
            labeled_idx, 
            unlabeled_idx, 
            labeled_batch_size, 
            unlabeled_batch_size):

        self.labeled_idx = labeled_idx
        self.unlabeled_idx = unlabeled_idx
        self.unlabeled_batch_size = unlabeled_batch_size
        self.labeled_batch_size = labeled_batch_size

        assert len(self.labeled_idx) >= self.labeled_batch_size > 0
        assert len(self.unlabeled_idx) >= self.unlabeled_batch_size > 0

    @property
    def num_labeled(self):
        return len(self.labeled_idx)

    def __iter__(self):
        # print("Balle balle")
        labeled_iter = iterate_once(self.labeled_idx)
        unlabeled_iter = iterate_eternally(self.unlabeled_idx)
        return (
            labeled_batch + unlabeled_batch
            for (labeled_batch, unlabeled_batch)
            in  zip(batch_iterator(labeled_iter, self.labeled_batch_size),
                    batch_iterator(unlabeled_iter, self.unlabeled_batch_size))
        )

    def __len__(self):
        return len(self.labeled_idx) // self.labeled_batch_size


def iterate_once(iterable):
    return np.random.permutation(iterable)


def iterate_eternally(indices):
    def infinite_shuffles():
        while True:
            yield np.random.permutation(indices)
    return itertools.chain.from_iterable(infinite_shuffles())


def batch_iterator(iterable, n):
    "Collect data into fixed-length chunks or blocks"
    args = [iter(iterable)] * n
    return zip(*args)


class TransformTwice:
    def __init__(self, transform):
        self.transform = transform

    def __call__(self, inp):
        out1 = self.transform(inp)
        out2 = self.transform(inp)
        return out1, out2

In [10]:
class Dataset():
    def __init__(self, config: dict):
        self.data_keys = set(['mnist', 'fashionmnist', 'cifar', 'svhn'])
        self.config = config
        self.labeled_ids = []
        self.unlabeled_ids = []
        self.image_tensors = []
        self.labels = []
        self.counter = 0
    
    def prepare(self, in_data='mnist', out_data=['cifar', 'svhn', 'fashionmnist'], indata_size=5000, outdata_size=1700, label_ratio=0.02):
        # Prepare OOD data
        for k in out_data:
            dataset = config[k]['dataset']
            transforms = TransformTwice(config[k]['transforms'])
            n_outlabels = int(outdata_size * label_ratio)
            dataset_ids = np.random.choice(len(dataset), outdata_size)
            dataset_labeled_ids = set(np.random.choice(dataset_ids, n_outlabels))
            # u, c = np.unique(dataset_labeled_ids, return_counts=True)
            # dup = u[c > 1]
            # print(dup)
            transform = torchvision.transforms.Compose([torchvision.transforms.PILToTensor()])
            print(transform(dataset[0][0]).shape)
            for idx in dataset_ids:
                img = dataset[idx][0]
                img_tensor = transforms(img)
                self.image_tensors.append(img_tensor)
                if idx in dataset_labeled_ids:
                    self.labeled_ids.append(self.counter)
                    self.labels.append(0)
                else:
                    self.unlabeled_ids.append(self.counter)
                    self.labels.append(-1)
                self.counter += 1
            print(len(self.labeled_ids))
            print(f'{k} dataset processed...')
        
        # Prepare ID data
        dataset = config[in_data]['dataset']
        transforms = TransformTwice(config[k]['transforms'])
        n_inlabels = int(indata_size * label_ratio)
        dataset_ids = np.random.choice(len(dataset), indata_size)
        dataset_labeled_ids = set(np.random.choice(dataset_ids, n_inlabels))
        for idx in dataset_ids:
            img = dataset[idx][0]
            img_tensor = transforms(img)
            self.image_tensors.append(img_tensor)
            if idx in dataset_labeled_ids:
                self.labeled_ids.append(self.counter)
                self.labels.append(1)
            else:
                self.unlabeled_ids.append(self.counter)
                self.labels.append(-1)
            self.counter += 1
        print(len(self.labeled_ids))
        print(f'{in_data} dataset processed...')
        
        random.shuffle(self.labeled_ids)
        random.shuffle(self.unlabeled_ids)
    
    def __len__(self):
        return len(self.image_tensors)
    
    def __getitem__(self, idx):
        return self.image_tensors[idx], self.labels[idx]

In [11]:
class SLDataset():
    def __init__(self, config: dict):
        self.data_keys = set(['mnist', 'fashionmnist', 'cifar', 'svhn'])
        self.config = config
        self.image_tensors = []
        self.labels = []
    
    def prepare(self, in_data='mnist', out_data=['cifar', 'svhn', 'fashionmnist'], indata_size=600, outdata_size=200):
        print(out_data)
        # Prepare OOD data
        for k in out_data:
            dataset = config[k]['dataset']
            transforms = config[k]['transforms']
            for i, (img, _) in enumerate(dataset):
                if i == outdata_size:
                    break
                img_tensor = transforms(img)
                self.image_tensors.append(img_tensor)
            self.labels += [0] * outdata_size
        
        # Prepare ID data
        dataset = config[in_data]['dataset']
        transforms = config[in_data]['transforms']
        for i, (img, _) in enumerate(dataset):
            if i == indata_size:
                break
            img_tensor = transforms(img)
            self.image_tensors.append(img_tensor)
        self.labels += [1] * indata_size
    
    def __len__(self):
        return len(self.image_tensors)
    
    def __getitem__(self, idx):
        return self.image_tensors[idx], self.labels[idx]

In [15]:
transform_to_three_channel = transforms.Lambda(lambda img: img.expand(3,*img.shape[1:]))

svhn_transforms = transforms.Compose([
                    # transforms.Grayscale(),
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Resize((32,32)),
                    transform_to_three_channel,
                ])

mnist_transforms = transforms.Compose([
                   transforms.RandomCrop(32, padding=4),
                   transforms.RandomHorizontalFlip(),
                   transforms.ToTensor(),
                   transforms.Resize((32,32)),
                   transform_to_three_channel
                ])

fashionmnist_transforms = transforms.Compose([
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Resize((32,32)),
                    transform_to_three_channel
                ])

cifar_transforms = transforms.Compose([
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    # transforms.Grayscale(),
                    transforms.ToTensor(),
                    transforms.Resize((32,32)),
                    transform_to_three_channel
                ])

food_transforms = transforms.Compose([
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    # transforms.Grayscale(),
                    transforms.ToTensor(),
                    transforms.Resize((32,32)),
                    transform_to_three_channel
                ])


In [16]:
celeb_transforms = transforms.Compose([
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    # transforms.Grayscale(),
                    transforms.ToTensor(),
                    transforms.Resize((32,32)),
                    transform_to_three_channel
                ])
stl10_transforms = transforms.Compose([
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    # transforms.Grayscale(),
                    transforms.ToTensor(),
                    transforms.Resize((32,32)),
                    transform_to_three_channel
                ])
german_sign_transforms = transforms.Compose([
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    # transforms.Grayscale(),
                    transforms.ToTensor(),
                    transforms.Resize((32,32)),
                    transform_to_three_channel
                ])
caltech_transforms = transforms.Compose([
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    # transforms.Grayscale(),
                    transforms.ToTensor(),
                    transforms.Resize((32,32)),
                    transform_to_three_channel
                ])

flowers_transforms = transforms.Compose([
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    # transforms.Grayscale(),
                    transforms.ToTensor(),
                    transforms.Resize((32,32)),
                    transform_to_three_channel
                ])

In [17]:
config = {}

config['svhn'] = {}
config['svhn']['dataset'] = svhn_dataset
config['svhn']['transforms'] = svhn_transforms

config['mnist'] = {}
config['mnist']['dataset'] = mnist_dataset
config['mnist']['transforms'] = mnist_transforms

config['fashionmnist'] = {}
config['fashionmnist']['dataset'] = fashionmnist_dataset
config['fashionmnist']['transforms'] = fashionmnist_transforms

config['cifar'] = {}
config['cifar']['dataset'] = cifar_dataset
config['cifar']['transforms'] = cifar_transforms

config['food'] = {}
config['food']['dataset'] = food_dataset
config['food']['transforms'] = food_transforms

config['stl10'] = {}
config['stl10']['dataset'] = stl_dataset
config['stl10']['transforms'] = stl10_transforms

config['flowers'] = {}
config['flowers']['dataset'] = flowers_dataset
config['flowers']['transforms'] = flowers_transforms

config['caltech'] = {}
config['caltech']['dataset'] = caltech_dataset
config['caltech']['transforms'] = caltech_transforms

config['german_sign'] = {}
config['german_sign']['dataset'] = german_sign_dataset
config['german_sign']['transforms'] = german_sign_transforms

In [18]:
ds = Dataset(config)
ds.prepare(in_data='cifar', out_data=['mnist', 'svhn', 'fashionmnist', 'stl10','food', 'caltech','flowers','german_sign'], indata_size=40000, outdata_size=5000, label_ratio=0.02)

torch.Size([1, 28, 28])
112
mnist dataset processed...
torch.Size([3, 32, 32])
220
svhn dataset processed...
torch.Size([1, 28, 28])
326
fashionmnist dataset processed...
torch.Size([3, 96, 96])
525
stl10 dataset processed...
torch.Size([3, 512, 512])
629
food dataset processed...
torch.Size([3, 337, 510])
784
caltech dataset processed...
torch.Size([3, 500, 754])
1299
flowers dataset processed...
torch.Size([3, 30, 29])
1427
german_sign dataset processed...
2860
cifar dataset processed...


In [19]:
len(ds.unlabeled_ids) + len(ds.labeled_ids)

80000

In [20]:
svhn_test_dataset = SVHN(root=data_dir, split='test', download=True)
mnist_test_dataset = MNIST(root=data_dir, train=False, download=True)
fashionmnist_test_dataset = FashionMNIST(root=data_dir, train=False, download=True)
cifar_test_dataset = CIFAR10(root=data_dir, train=False, download=True)

Using downloaded and verified file: /scratch/gg2501/ML_Project/data/test_32x32.mat
Files already downloaded and verified


In [21]:
stl_test_dataset = STL10(root=data_dir, split='test', download=True)
food_test_dataset = Food101(root=data_dir, split='test', download = True)
# celeb_dataset = CelebA(root=data_dir, download = True)
flowers_test_dataset = Flowers102(root=data_dir, split='test', download = True)
caltech_test_dataset = Caltech101(root=data_dir, download = True)
german_sign_test_dataset = GTSRB(root=data_dir, split='test', download = True)

Files already downloaded and verified
Files already downloaded and verified


In [22]:
test_config = {}

test_config['svhn'] = {}
test_config['svhn']['dataset'] = svhn_test_dataset
test_config['svhn']['transforms'] = svhn_transforms

test_config['mnist'] = {}
test_config['mnist']['dataset'] = mnist_test_dataset
test_config['mnist']['transforms'] = mnist_transforms

test_config['fashionmnist'] = {}
test_config['fashionmnist']['dataset'] = fashionmnist_test_dataset
test_config['fashionmnist']['transforms'] = fashionmnist_transforms

test_config['cifar'] = {}
test_config['cifar']['dataset'] = cifar_test_dataset
test_config['cifar']['transforms'] = cifar_transforms

test_config['food'] = {}
test_config['food']['dataset'] = food_test_dataset
test_config['food']['transforms'] = food_transforms

test_config['stl10'] = {}
test_config['stl10']['dataset'] = stl_test_dataset
test_config['stl10']['transforms'] = stl10_transforms

test_config['flowers'] = {}
test_config['flowers']['dataset'] = flowers_test_dataset
test_config['flowers']['transforms'] = flowers_transforms

test_config['caltech'] = {}
test_config['caltech']['dataset'] = caltech_test_dataset
test_config['caltech']['transforms'] = caltech_transforms

test_config['german_sign'] = {}
test_config['german_sign']['dataset'] = german_sign_test_dataset
test_config['german_sign']['transforms'] = german_sign_transforms

In [23]:
test_ds = SLDataset(test_config)
test_ds.prepare(in_data='cifar', out_data=['mnist', 'svhn', 'fashionmnist', 'stl10','food', 'caltech','flowers','german_sign'], indata_size=2000, outdata_size=250)

['mnist', 'svhn', 'fashionmnist', 'stl10', 'food', 'caltech', 'flowers', 'german_sign']


In [24]:
len(test_ds)

4000

In [25]:
train_batch_sampler = LabeledUnlabeledBatchSampler(ds.labeled_ids, ds.unlabeled_ids, 32, 32)
trainloader = torch.utils.data.DataLoader(ds, batch_sampler=train_batch_sampler, pin_memory=True)
testloader = torch.utils.data.DataLoader(test_ds, batch_size=64, shuffle=True, pin_memory=True)

In [26]:
for batch in trainloader:
    # print(len(batch))
    print(batch[0][0].shape)
    print(batch[0][1].shape)
    print(batch[1].shape)
    # cv2.imshow('image window1', batch[0][0])
    # cv2.imshow('image window2', batch[1][0])
    break

torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64])


In [27]:
for batch in testloader:
    print(len(batch))
    print(batch[0].get_device())
    print(batch[0].shape)
    print(batch[1].shape)
    break

2
-1
torch.Size([64, 3, 32, 32])
torch.Size([64])


In [28]:
import torch.nn as nn

cls_num_list = [6000, 45000]

class LDAMLoss(nn.Module):
    
    def __init__(self, cls_num_list, max_m=0.5, weight=None, s=30):
        super(LDAMLoss, self).__init__()
        m_list = 1.0 / np.sqrt(np.sqrt(cls_num_list))
        m_list = m_list * (max_m / np.max(m_list))
        m_list = torch.cuda.FloatTensor(m_list)
        self.m_list = m_list
        assert s > 0
        self.s = s
        self.weight = weight

    def forward(self, x, target):
        index = torch.zeros_like(x, dtype=torch.uint8)
        index.scatter_(1, target.data.view(-1, 1), 1)
        
        index_float = index.type(torch.cuda.FloatTensor)
        batch_m = torch.matmul(self.m_list[None, :], index_float.transpose(0,1))
        batch_m = batch_m.view((-1, 1))
        x_m = x - batch_m
    
        output = torch.where(index, x_m, x)
        return F.cross_entropy(self.s*output, target, weight=self.weight)

In [29]:
img_shape = (3, 32, 32)
flow = 'ResidualFlow'
model_cfg = getattr(flow_ssl, flow)
net = model_cfg(in_channels=img_shape[0], num_classes=2)

In [30]:
if flow in ["iCNN3d", "iResnetProper","SmallResidualFlow","ResidualFlow","MNISTResidualFlow"]:
    net = net.flow

In [31]:
from experiments.train_flows.utils import train_utils

means = 'random'
means_r = 1.0
cov_std = 1.0
# img_shape = (1, 32, 32)
device = 'cuda'
n_classes = 2

net = net.to(device)
r = means_r
cov_std = torch.ones((n_classes)) * cov_std
cov_std = cov_std.to(device)
means = train_utils.get_means(means, num_means=n_classes, r=means_r, trainloader=trainloader, 
                        shape=img_shape, device=device, net=net)
means_init = means.clone().detach()

In [32]:
from scipy.spatial.distance import cdist

print("Means:", means)
print("Cov std:", cov_std)
means_np = means.cpu().numpy()
print("Pairwise dists:", cdist(means_np, means_np))

Means: tensor([[-0.5294,  0.7078, -0.9747,  ...,  0.2047, -0.2061, -2.2594],
        [-0.3701, -0.8483,  2.1468,  ..., -0.8181,  0.4732,  1.2964]],
       device='cuda:0')
Cov std: tensor([1., 1.], device='cuda:0')
Pairwise dists: [[ 0.         76.90869991]
 [76.90869991  0.        ]]


In [33]:
from flow_ssl.distributions import SSLGaussMixture
from flow_ssl import FlowLoss

means_trainable = True
covs_trainable = True
weights_trainable = True

if means_trainable:
    print("Using learnable means")
    means = torch.tensor(means_np, requires_grad=True, device=device)

prior = SSLGaussMixture(means, device=device)
prior.weights.requires_grad = weights_trainable
prior.inv_cov_stds.requires_grad = covs_trainable
loss_fn = FlowLoss(prior)
sup_loss_fn = LDAMLoss(cls_num_list, s=15)

Using learnable means


In [34]:
from experiments.train_flows.utils import norm_util
import torch.optim as optim

param_groups = norm_util.get_param_groups(net, 0.0, norm_suffix='weight_g')

optimizer = optim.Adam(param_groups, lr=5e-4, weight_decay=1e-2)
opt_gmm = optim.Adam([prior.means, prior.weights, prior.inv_cov_stds], lr=1e-4, weight_decay=1e-2)

In [35]:
from tensorboardX import SummaryWriter

writer = SummaryWriter(log_dir='./')
device = 'cuda' if torch.cuda.is_available() and len([0]) > 0 else 'cpu'
start_epoch = 0

In [36]:
def test_classifier(epoch, net, testloader, device, loss_fn, writer=None, postfix="",
                    show_classification_images=False, confusion=False):
    net.eval()
    loss_meter = shell_util.AverageMeter()
    jaclogdet_meter = shell_util.AverageMeter()
    acc_meter = shell_util.AverageMeter()
    all_pred_labels = []
    all_xs = []
    all_ys = []
    all_zs = []
    with torch.no_grad():
        with tqdm(total=len(testloader.dataset)) as progress_bar:
            for x, y in testloader:
                all_xs.append(x.data.numpy())
                all_ys.append(y.data.numpy())
                x = x.to(device)
                y = y.to(device)
                z = net(x)
                all_zs.append(z.cpu().data.numpy())
                sldj = net.logdet()
                loss = loss_fn(z, y=y, sldj=sldj)
                loss_meter.update(loss.item(), x.size(0))
                jaclogdet_meter.update(sldj.mean().item(), x.size(0))

                preds = loss_fn.prior.classify(z.reshape((len(z), -1)))
                preds = preds.reshape(y.shape)
                all_pred_labels.append(preds.cpu().data.numpy())
                acc = (preds == y).float().mean().item()
                acc_meter.update(acc, x.size(0))

                progress_bar.set_postfix(loss=loss_meter.avg,
                                     bpd=optim_util.bits_per_dim(x, loss_meter.avg),
                                     acc=acc_meter.avg)
                progress_bar.update(x.size(0))
    all_pred_labels = np.hstack(all_pred_labels)
    all_xs = np.vstack(all_xs)
    all_zs = np.vstack(all_zs)
    all_ys = np.hstack(all_ys)

    if writer is not None:
        writer.add_scalar("test/loss{}".format(postfix), loss_meter.avg, epoch)
        writer.add_scalar("test/acc{}".format(postfix), acc_meter.avg, epoch)
        writer.add_scalar("test/bpd{}".format(postfix), optim_util.bits_per_dim(x, loss_meter.avg), epoch)
        writer.add_scalar("test/jaclogdet{}".format(postfix), jaclogdet_meter.avg, epoch)

        for cls in range(np.max(all_pred_labels)+1):
            num_imgs_cls = (all_pred_labels==cls).sum()
            writer.add_scalar("test_clustering/num_class_{}_{}".format(cls,postfix), 
                    num_imgs_cls, epoch)
            if num_imgs_cls == 0:
                writer.add_scalar("test_clustering/num_class_{}_{}".format(cls,postfix), 
                    0., epoch)
                continue
            writer.add_histogram('label_distributions/num_class_{}_{}'.format(cls,postfix), 
                    all_ys[all_pred_labels==cls], epoch)

            writer.add_histogram(
                'distance_distributions/num_class_{}'.format(cls),
                torch.norm(torch.tensor(all_zs[all_pred_labels==cls]) - loss_fn.prior.means[cls].cpu(), p=2, dim=1),
                epoch
            )

            if show_classification_images:
                images_cls = all_xs[all_pred_labels==cls][:10]
                images_cls = torch.from_numpy(images_cls).float()
                images_cls_concat = torchvision.utils.make_grid(
                        images_cls, nrow=2, padding=2, pad_value=255)
                writer.add_image("test_clustering/class_{}".format(cls), 
                        images_cls_concat)

        if confusion:
            fig = plt.figure(figsize=(8, 8))
            cm = confusion_matrix(all_ys, all_pred_labels)
            cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
            sns.heatmap(cm, annot=True, cmap=plt.cm.Blues)
            plt.ylabel('True label')
            plt.xlabel('Predicted label')
            conf_img = torch.tensor(np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep=''))
            conf_img = torch.tensor(conf_img.reshape(fig.canvas.get_width_height()[::-1] + (3,))).transpose(0, 2).transpose(1, 2)
            writer.add_image("confusion", conf_img, epoch)

In [37]:
total_labels = len(ds.labeled_ids)

In [38]:
def sample(net, prior, batch_size, cls, device, sample_shape):
    """Sample from RealNVP model.
    Args:
        net (torch.nn.DataParallel): The RealNVP model wrapped in DataParallel.
        batch_size (int): Number of samples to generate.
        device (torch.device): Device to use.
    """
    with torch.no_grad():
        if cls is not None:
            z = prior.sample((batch_size,), gaussian_id=cls)
        else:
            z = prior.sample((batch_size,))
        x = net.inverse(z)

        return x

In [39]:
from experiments.train_flows.utils import optim_util

def train(epoch, net, trainloader, device, optimizer, opt_gmm, loss_fn,
          label_weight, max_grad_norm, consistency_weight,
          writer, sup_loss_fn, use_unlab=True,  acc_train_all_labels=False,
          ):
    print('\nEpoch: %d' % epoch)
    net.train()
    loss_meter = shell_util.AverageMeter()
    loss_unsup_meter = shell_util.AverageMeter()
    loss_nll_meter = shell_util.AverageMeter()
    loss_consistency_meter = shell_util.AverageMeter()
    jaclogdet_meter = shell_util.AverageMeter()
    acc_meter = shell_util.AverageMeter()
    acc_all_meter = shell_util.AverageMeter()
    with tqdm(total=2*total_labels) as progress_bar:
        for (x1, x2), y in trainloader:

            x1 = x1.to(device)
            if not acc_train_all_labels:
                y = y.to(device)
            else:
                y, y_all_lab = y[:, 0], y[:, 1]
                y = y.to(device)
                y_all_lab = y_all_lab.to(device)

            labeled_mask = (y != NO_LABEL)

            optimizer.zero_grad()
            opt_gmm.zero_grad()

            with torch.no_grad():
                x2 = x2.to(device)
                # z21 = net(x2)
                # z22 = net(transform(x2))
                # y22 = classify(z22)
                # l_cons = FlowLoss(z21, y22)
                # print(x2.get_device(), next(net.parameters()).is_cuda)
                z2 = net(x2)
                z2 = z2.detach()
                pred2 = loss_fn.prior.classify(z2.reshape((len(z2), -1)))

            z1 = net(x1)
            sldj = net.logdet()

            z_all = z1.reshape((len(z1), -1))
            z_labeled = z_all[labeled_mask]
            y_labeled = y[labeled_mask]

            logits_all = loss_fn.prior.class_logits(z_all)
            logits_labeled = logits_all[labeled_mask]
            loss_nll = F.cross_entropy(logits_labeled, y_labeled)
            # loss_nll = sup_loss_fn(logits_labeled, y_labeled)
            # print(loss_nll)
            # loss_nll = loss_nll.mean()
            # print(loss_nll)

            if use_unlab:
                loss_unsup = loss_fn(z1, sldj=sldj)
                # print(loss_unsup)
                loss = loss_nll * label_weight + loss_unsup
            else:
                loss_unsup = torch.tensor([0.])
                loss = loss_nll

            # consistency loss
            loss_consistency = loss_fn(z1, sldj=sldj, y=pred2)
            loss = loss + loss_consistency * consistency_weight

            loss.backward()
            optim_util.clip_grad_norm(optimizer, max_grad_norm)
            optimizer.step()
            opt_gmm.step()

            preds_all = torch.argmax(logits_all, dim=1)
            preds = preds_all[labeled_mask]
            acc = (preds == y_labeled).float().mean().item()
            if acc_train_all_labels:
                acc_all = (preds_all == y_all_lab).float().mean().item()
            else:
                acc_all = acc

            acc_meter.update(acc, x1.size(0))
            acc_all_meter.update(acc_all, x1.size(0))
            loss_meter.update(loss.item(), x1.size(0))
            loss_unsup_meter.update(loss_unsup.item(), x1.size(0))
            loss_nll_meter.update(loss_nll.item(), x1.size(0))
            jaclogdet_meter.update(sldj.mean().item(), x1.size(0))
            loss_consistency_meter.update(loss_consistency.item(), x1.size(0))

            progress_bar.set_postfix(loss=loss_meter.avg,
                                     bpd=optim_util.bits_per_dim(x1, loss_unsup_meter.avg),
                                     acc=acc_meter.avg,
                                     acc_all=acc_all_meter.avg)
            progress_bar.update(y_labeled.size(0))

    x1_img = torchvision.utils.make_grid(x1[:10], nrow=2 , padding=2, pad_value=255)
    x2_img = torchvision.utils.make_grid(x2[:10], nrow=2 , padding=2, pad_value=255)
    writer.add_image("data/x1", x1_img)
    writer.add_image("data/x2", x2_img)

    writer.add_scalar("train/loss", loss_meter.avg, epoch)
    writer.add_scalar("train/loss_unsup", loss_unsup_meter.avg, epoch)
    writer.add_scalar("train/loss_nll", loss_nll_meter.avg, epoch)
    writer.add_scalar("train/jaclogdet", jaclogdet_meter.avg, epoch)
    writer.add_scalar("train/acc", acc_meter.avg, epoch)
    writer.add_scalar("train/acc_all", acc_all_meter.avg, epoch)
    writer.add_scalar("train/bpd", optim_util.bits_per_dim(x1, loss_unsup_meter.avg), epoch)
    writer.add_scalar("train/loss_consistency", loss_consistency_meter.avg, epoch)

In [40]:
from experiments.train_flows.utils import shell_util

from tqdm.notebook import tqdm
import torch.nn as nn
import math
import torch.nn.init as init
import torch.nn.functional as F

NO_LABEL = -1
schedule = None
n_epochs = 10
lr = 5e-4
lr_gmm = 1e-4
consistency_weight = 0.01
consistency_rampup = 1
label_weight = 1.0
max_grad_norm = 100.0
save_freq = 2
ckptdir = './'
eval_freq = 2
confusion = True
num_samples = 50

def linear_rampup(final_value, epoch, num_epochs, start_epoch=0):
    t = (epoch - start_epoch + 1) / num_epochs
    if t > 1:
        t = 1.
    return t * final_value

for epoch in range(start_epoch, n_epochs):
    cons_weight = linear_rampup(consistency_weight, epoch, consistency_rampup, start_epoch)
    
    writer.add_scalar("hypers/learning_rate", lr, epoch)
    writer.add_scalar("hypers/learning_rate_gmm", lr_gmm, epoch)
    writer.add_scalar("hypers/consistency_weight", cons_weight, epoch)

    train(epoch, net, trainloader, device, optimizer, opt_gmm, loss_fn,
          label_weight, max_grad_norm, cons_weight,
          writer, sup_loss_fn, use_unlab=True)

    # Save checkpoint
    if (epoch % save_freq == 0):
        print('Saving...')
        state = {
            'net': net.state_dict(),
            'epoch': epoch,
            'means': prior.means,
        }
        os.makedirs(ckptdir, exist_ok=True)
        torch.save(state, os.path.join(ckptdir, str(epoch)+'.pt'))

    # Save samples and data
    if epoch % eval_freq == 0:
        test_classifier(epoch, net, testloader, device, loss_fn, writer, confusion=confusion)
        # if args.swa:
        #     optimizer.swap_swa_sgd() 
        #     print("updating bn")
        #     SWA.bn_update(bn_loader, net)
        #     utils.test_classifier(epoch, net, testloader, device, loss_fn, 
        #             writer, postfix="_swa")

        z_means = prior.means
        data_means = net.inverse(z_means)
        z_mean_imgs = torchvision.utils.make_grid(
                z_means.reshape((n_classes, *img_shape)), nrow=2)
        data_mean_imgs = torchvision.utils.make_grid(
                data_means.reshape((n_classes, *img_shape)), nrow=2)
        writer.add_image("z_means", z_mean_imgs, epoch)
        writer.add_image("data_means", data_mean_imgs, epoch)

        means_np = prior.means.detach().cpu().numpy()
        fig = plt.figure(figsize=(8, 8))
        sns.heatmap(cdist(means_np, means_np))
        img_data = torch.tensor(np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep=''))
        img_data = torch.tensor(img_data.reshape(fig.canvas.get_width_height()[::-1] + (3,))).transpose(0, 2).transpose(1, 2)
        writer.add_image("mean_dists", img_data, epoch)

        for i in range(n_classes):
            writer.add_scalar("train_gmm/weight/{}".format(i), F.softmax(prior.weights)[i], epoch)

        for i in range(n_classes):
            writer.add_scalar("train_gmm/cov/{}".format(i), F.softplus(prior.inv_cov_stds[i])**2, epoch)

        for i in range(n_classes):
            writer.add_scalar("train_gmm/mean_dist_init/{}".format(i), torch.norm(prior.means[i]-means_init[i], 2), epoch)

        images = []
        for i in range(n_classes):
            images_cls = sample(net, loss_fn.prior, num_samples // n_classes,
                                      cls=i, device=device, sample_shape=img_shape)
            images.append(images_cls)
            images_cls_concat = torchvision.utils.make_grid(
                    images_cls, nrow=2, padding=2, pad_value=255)
            writer.add_image("samples/class_"+str(i), images_cls_concat)
        images = torch.cat(images)
        os.makedirs(os.path.join(ckptdir, 'samples'), exist_ok=True)
        images_concat = torchvision.utils.make_grid(images, nrow=num_samples //  n_classes , padding=2, pad_value=255)
        os.makedirs(ckptdir, exist_ok=True)
        torchvision.utils.save_image(images_concat, 
                                    os.path.join(ckptdir, 'samples/epoch_{}.png'.format(epoch)))


Epoch: 0


  0%|          | 0/5720 [00:00<?, ?it/s]

/scratch/gg2501/ML_Project/flowgmm-public/flow_ssl/distributions.py:62: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  log_probs_weighted = log_probs + torch.log(F.softmax(self.weights))
/scratch/gg2501/ML_Project/flowgmm-public/flow_ssl/distributions.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  mixture_log_probs = torch.logsumexp(all_log_probs + torch.log(F.softmax(self.weights)), dim=1)


Saving...


  0%|          | 0/4000 [00:00<?, ?it/s]

...

  0%|          | 0/4000 [00:00<?, ?it/s]

/state/partition1/job-28042615/ipykernel_89665/1258797901.py:78: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  conf_img = torch.tensor(np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep=''))
/state/partition1/job-28042615/ipykernel_89665/1258797901.py:79: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  conf_img = torch.tensor(conf_img.reshape(fig.canvas.get_width_height()[::-1] + (3,))).transpose(0, 2).transpose(1, 2)
/state/partition1/job-28042615/ipykernel_89665/3055084958.py:74: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  img_data = torch.tensor(np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep=''))
/state/partition1/job-28042615/ipyker


Epoch: 9


  0%|          | 0/5720 [00:00<?, ?it/s]

/scratch/gg2501/ML_Project/flowgmm-public/flow_ssl/distributions.py:62: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  log_probs_weighted = log_probs + torch.log(F.softmax(self.weights))
/scratch/gg2501/ML_Project/flowgmm-public/flow_ssl/distributions.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  mixture_log_probs = torch.logsumexp(all_log_probs + torch.log(F.softmax(self.weights)), dim=1)


In [ ]:
len(ds.labeled_ids)

In [ ]:
resume = './4.pt'
print('Resuming from checkpoint at', resume)
checkpoint = torch.load(resume)
# net.load_state_dict(checkpoint['net'])
# start_epoch = checkpoint['epoch']

In [ ]:
net.load_state_dict(checkpoint['net'])
start_epoch = checkpoint['epoch']

In [42]:
omniglot_transforms = transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Resize((32,32)),
                      transform_to_three_channel
                    ])

omniglot_dataset = Omniglot(root=data_dir, background=True, download=True, transform=omniglot_transforms)

  0%|          | 0/9464212 [00:00<?, ?it/s]

Extracting /scratch/gg2501/ML_Project/data/omniglot-py/images_background.zip to /scratch/gg2501/ML_Project/data/omniglot-py


In [43]:
first, second = torch.utils.data.random_split(omniglot_dataset, [10000, 9280])
omniglot_loader = torch.utils.data.DataLoader(first, batch_size=64, shuffle=True, pin_memory=True)

In [ ]:
all_xs = []
all_ys = []
all_zs = []
all_pred_labels = []

net.eval()
with torch.no_grad():
    with tqdm(total=len(omniglot_loader.dataset)) as progress_bar:
        for x, y in omniglot_loader:
            all_xs.append(x.data.numpy())
            all_ys.append(y.data.numpy())
            x = x.to(device)
            y = y.to(device)
            z = net(x)
            all_zs.append(z.cpu().data.numpy())
            sldj = net.logdet()

            preds = loss_fn.prior.classify(z.reshape((len(z), -1)))
            preds = preds.reshape(y.shape)
            all_pred_labels.append(preds.cpu().data.numpy())
            acc = (preds == y).float().mean().item()
            progress_bar.update(x.size(0))

In [ ]:
(np.hstack(all_pred_labels) == 1).sum()

In [ ]:
test_ds = SLDataset(test_config)
test_ds.prepare(in_data='cifar', indata_size=1000, outdata_size=300)

In [ ]:
mnist_loader = torch.utils.data.DataLoader(mnist_test_dataset, batch_size=64, shuffle=True, pin_memory=True)

In [ ]:
all_xs = []
all_ys = []
all_zs = []
all_pred_labels = []

net.eval()
with torch.no_grad():
    with tqdm(total=len(mnist_loader.dataset)) as progress_bar:
        for x, y in mnist_loader:
            all_xs.append(x.data.numpy())
            all_ys.append(y.data.numpy())
            x = x.to(device)
            y = y.to(device)
            z = net(x)
            all_zs.append(z.cpu().data.numpy())
            sldj = net.logdet()

            preds = loss_fn.prior.classify(z.reshape((len(z), -1)))
            preds = preds.reshape(y.shape)
            all_pred_labels.append(preds.cpu().data.numpy())
            acc = (preds == y).float().mean().item()
            progress_bar.update(x.size(0))

In [ ]:
(np.hstack(all_pred_labels) == all_ys).sum()

In [ ]:
all_ys = np.zeros((10000))

In [ ]:
all_ys

In [ ]:
from sklearn.metrics import auc, precision_recall_curve, roc_curve

def auroc(preds, labels, pos_label=1):
    """Calculate and return the area under the ROC curve using unthresholded predictions on the data and a binary true label.
    
    preds: array, shape = [n_samples]
           Target normality scores, can either be probability estimates of the positive class, confidence values, or non-thresholded measure of decisions.
           i.e.: an high value means sample predicted "normal", belonging to the positive class
           
    labels: array, shape = [n_samples]
            True binary labels in range {0, 1} or {-1, 1}.
    pos_label: label of the positive class (1 by default)
    """
    fpr, tpr, _ = roc_curve(labels, preds, pos_label=pos_label)
    return auc(fpr, tpr)

In [ ]:
auroc(np.hstack(all_pred_labels), all_ys)

In [ ]:
class TestDataset():
    def __init__(self, config: dict):
        self.data_keys = set(['mnist', 'fashionmnist', 'cifar', 'svhn'])
        self.config = config
        self.image_tensors = []
        self.labels = []
    
    def prepare(self, in_data='cifar', out_data=['svhn'], indata_size=600, outdata_size=600):
        # Prepare OOD data
        s = outdata_size // len(out_data)
        for k in out_data:
            dataset = config[k]['dataset']
            transforms = config[k]['transforms']
            for i, (img, _) in enumerate(dataset):
                if i == outdata_size:
                    break
                img_tensor = transforms(img)
                self.image_tensors.append(img_tensor)
            self.labels += [0] * outdata_size
        
        # Prepare ID data
        dataset = config[in_data]['dataset']
        transforms = config[in_data]['transforms']
        for i, (img, _) in enumerate(dataset):
            if i == indata_size:
                break
            img_tensor = transforms(img)
            self.image_tensors.append(img_tensor)
        self.labels += [1] * indata_size
    
    def __len__(self):
        return len(self.image_tensors)
    
    def __getitem__(self, idx):
        return self.image_tensors[idx], self.labels[idx]

In [ ]:
testds = TestDataset(test_config)
testds.prepare(indata_size=1000, outdata_size=1000)

In [ ]:
test_loader = torch.utils.data.DataLoader(testds, batch_size=64, shuffle=True, pin_memory=True)

In [ ]:
resume = './8.pt'
print('Resuming from checkpoint at', resume)
checkpoint = torch.load(resume)
net.load_state_dict(checkpoint['net'])
start_epoch = checkpoint['epoch']